In [101]:
from ase.calculators.castep import Castep
from ase.visualize import view
import ase
import os
import numpy as np
import warnings
warnings.simplefilter('ignore')
from glob import glob
import re

In [100]:
def get_MP_grid(atoms_in, kspacing: float = 0.01):
    A = np.array(atoms_in.cell)
    x= A[0,:] ; y= A[1,:] ;z= A[2,:]
    V = atoms_in.get_volume()
    ix = (np.pi/V) *np.cross(y, z)
    iy = (np.pi/V) *np.cross(z, x)
    iz = (np.pi/V) *np.cross(x, y)
    mx, my, mz =int(np.round((1/3)*np.linalg.norm(ix)/kspacing) -1), \
            int(np.round((1/3)*np.linalg.norm(iy)/kspacing) -1), \
            int(np.round((1/3)*np.linalg.norm(iz)/kspacing)-1)
    return [mx, my, mz]

[6.61260611e-04 6.12040064e-04 2.56794893e+00]
[19.13921587 18.52014034  2.4467784  90.01372417 90.1327905  89.92431985]


In [102]:
files = ['BeC_tube+wire.geom',  'Be_nanowire.xyz',  'C_nanotube.xyz']
atoms = ase.io.read(f'geom_opti/{files[0]}')
view(atoms*[1,1,20], viewer='nglview')

In [71]:
!ls geom_opti/

BeC_tube+wire.castep  Be_nanowire.castep


In [61]:
param_string = "TASK: SinglePoint \n" \
"CUT_OFF_ENERGY: 1000 \n" \
"DATA_DISTRIBUTION: kpoint \n" \
"OPT_STRATEGY: speed \n" \
"XC_FUNCTIONAL: PBE \n" \
"GRID_SCALE: 2 \n" \
"FINE_GRID_SCALE: 2.0 \n" \
"FIX_OCCUPANCY: false \n" \
"METALS_METHOD: dm \n" \
"MIXING_SCHEME: pulay \n" \
"NEXTRA_BANDS: 35 \n" \
"SMEARING_WIDTH: 0.05 ev \n" \
"SPIN_POLARISED: false \n" \
"CALCULATE_STRESS : TRUE \n" \
" \n" \
"#%BLOCK devel_code \n" \
"#PARALLEL: kpoint=64 : ENDPARALLEL \n" \
"#%ENDBLOCK devel_code \n"

calculator= Castep()
calculator.cell.kpoint_mp_grid =  '1 1 500'
calculator.cell.fix_all_cell = 'true'

In [64]:
files = ['BeC_tube+wire.xyz',  'Be_nanowire.xyz',  'C_nanotube.xyz']
atoms = ase.io.read(f'../unopt_xyzs/{files[2]}')
atoms.set_calculator(calculator)
ase.io.write('convergence_study/C_nanotube.cell', atoms , format='castep-cell')
f = open('convergence_study/C_nanotube.param', "w")
f.write(param_string)
f.close()


In [57]:
A = list(np.sort(glob('convergence_study/*.castep')))
labels= sum([re.findall("\d+", a) for a in A], [])
atoms = [ase.io.read(a) for a in A]
for i, a in enumerate(atoms[0:4]):
    F = atoms[4].get_forces() - a.get_forces()
    print("Ecut = ", int(labels[i]), 'Mean dF', np.linalg.norm(F, axis=1).mean())

Ecut =  800 Mean dF 0.010542956759381291
Ecut =  1000 Mean dF 0.005403544805278422
Ecut =  1200 Mean dF 0.0038193203045597296
Ecut =  1400 Mean dF 0.008507855668180167


In [80]:
files = ['BeC_tube+wire.geom']
atoms = ase.io.read(f'geom_opti/{files[0]}')

ase.io.write('phonons/BeC.cell', atoms , format='castep-cell')

True